In [2]:
import sys  
sys.path.insert(1, '/home/sanjay/Research/MCTS/Codes/pda_simulator/notebooks')

import math
import numpy as np
import pandas as pd

from ipynb.fs.full.config import Config

import warnings
warnings.filterwarnings('ignore')

In [4]:
config = Config()

In [5]:
class LimitPriceDeterminants:
  def __init__(self, limitPrice, profitMargin, executionTimeslot):
    
    self.limitPrice = limitPrice
    self.profitMargin = profitMargin
    self.delta = 0.0
    self.executionTimeslot = executionTimeslot

    self.learningRate = 0.8
    self.momentumCoefficient = 0.5

In [ ]:
class ZIP:

    def __init__(self, total_demand, number_of_bids=1, buy_limit_price_min=-100.0, buy_limit_price_max=-1.0, sell_limit_price_min=0.5, sell_limit_price_max=100.0, id='ZIP'):
        self.seed_value = 0 # seed value will be overidden
        self.id = id
        self.total_demand = total_demand 
        self.cleared_demand = 0
        self.min_bid_quant = 0.01
        self.number_of_bids = number_of_bids
        self.buy_limit_price_min = buy_limit_price_min
        self.buy_limit_price_max = buy_limit_price_max
        self.sell_limit_price_min = sell_limit_price_min
        self.sell_limit_price_max = sell_limit_price_max

        self.bid_price_by_future_timeslot = dict()
        self.ask_price_by_future_timeslot = dict()

    def gen_function(self, rem_quantity):
        price = np.random.uniform(self.low, self.high)
        quantity = max(rem_quantity/self.number_of_bids, self.min_bid_quant)
        return price, quantity
    
    def set_cleared_demand(self, cleared_demand):
        self.cleared_demand += cleared_demand



    def bids(self, timeslot, currentTimeslot):

        rem_quantity = self.total_demand - self.cleared_demand
        
        if rem_quantity < self.min_bid_quant:
            return None
        
        amount_needed = rem_quantity
        remaining_tries = timeslot - currentTimeslot

        bids = list()
        if remaining_tries > 0:
            
            if amount_needed > 0.0:
                
                if timeslot not in self.bid_price_by_future_timeslot.keys():
                    lpd = LimitPriceDeterminants(np.random.random()*self.buy_limit_price_min, -0.1 , currentTimeslot)
                    self.bid_price_by_future_timeslot.update({timeslot: lpd})
                    bid = lpd.limitPrice
                else:
                    last_mcp = 40                                                   # last mcp in the previous auction instance
                    sdelta = lpd.learningRate * (-lastMCP - lpd.limitPrice)         # For bid, limitPrice is negetive, so "-lastMCP"
                    lpd.executionTimeslot = currentTimeslot
                    lpd.delta = lpd.momentumCoefficient * lpd.delta + (1 - lpd.momentumCoefficient) * sdelta
                    lpd.profitMargin = max(-1.0, lpd.profitMargin + (lpd.delta/lpd.limitPrice))
                    lpd.limitPrice = max(config.buy_limit_price_min, lpd.limitPrice * (1 + lpd.profitMargin))
                    self.bid_price_by_future_timeslot.update({timeslot: lpd})
                    bid = lpd.limitPrice
                
            else:
                
                if timeslot not in self.ask_price_by_future_timeslot.keys():
                    lpd = LimitPriceDeterminants((config.sell_limit_price_max + config.sell_limit_price_min)/2, 0.1, currentTimeslot)
                    self.ask_price_by_future_timeslot.update({timeslot: lpd})
                    bid = lpd.limitPrice
                else:
                    last_mcp = 40                                                   # last mcp in the previous auction instance
                    sdelta = lpd.learningRate * (lastMCP - lpd.limitPrice) 
                    lpd.executionTimeslot = currentTimeslot
                    lpd.delta = lpd.momentumCoefficient * lpd.delta + (1 - lpd.momentumCoefficient) * sdelta
                    lpd.profitMargin = max(-1.0, lpd.profitMargin + (lpd.delta/lpd.limitPrice))
                    lpd.limitPrice = max(config.sell_limit_price_max, lpd.limitPrice * (1 + lpd.profitMargin))
                    self.ask_price_by_future_timeslot.update({timeslot: lpd})
                    bid = lpd.limitPrice

            bids.append([self.id, bid, amount_needed])
        else:
            bids.append([self.id, -config.market_order_bid_price, amount_needed])

        return bids